In [1]:
%cd ..

/environment/Projects/dl


In [2]:
! python -m pip install deps/configfy-build/

Processing ./deps/configfy-build
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for configfy-build: filename=configfy_build-0.0.1-py3-none-any.whl size=2954 sha256=11908df93e94a23261f489de55b64f755d5fd2c581f73fd6095bcd59bbc27122
  Stored in directory: /home/stefan-cristianhantig/.cache/pip/wheels/18/58/ab/c7d510ca549c784c51bd5f5aa15d5d254efc9c365944543f60
Successfully built configfy-build
  Attempting uninstall: configfy-build
    Found existing installation: configfy-build 0.0.1
    Uninstalling configfy-build-0.0.1:
      Successfully uninstalled configfy-build-0.0.1


In [2]:
!python -m pip list | grep torch

light-torch              0.0.1
torch                    1.13.1
torchsummary             1.5.1
torchvision              0.14.1


In [3]:
! python -m pip install deps/light-torch/

Processing ./deps/light-torch
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for light-torch: filename=light_torch-0.0.1-py3-none-any.whl size=4741 sha256=85186387c6b3715842a276e87ede1b92cefa5fe2d161928d45b3e46cec7745a8
  Stored in directory: /home/stefan-cristianhantig/.cache/pip/wheels/76/4b/a5/ce53c36ade717c024783dce5309d843ab5a592d06a292dafc2
Successfully built light-torch
  Attempting uninstall: light-torch
    Found existing installation: light-torch 0.0.1
    Uninstalling light-torch-0.0.1:
      Successfully uninstalled light-torch-0.0.1


In [4]:
! python -m pip install torchsummary

In [2]:
from src.object_detection.yolov1 import YoloV1Backbone, YoloV1

In [3]:
from src.classification.models.resnet import Resnet

In [4]:
from torchsummary import summary

In [5]:
import json

In [6]:
ANNOTATION_FILE = "resources/object_detection/annotations/instances_train2014.json"

In [7]:
with open(ANNOTATION_FILE, "r") as json_file:
    coco = json.load(json_file)

In [8]:
import cv2

In [9]:
from pathlib import Path

In [10]:
import matplotlib.pyplot as plt

In [11]:
IMAGES_PATH = Path("./resources/object_detection/train2014/")

In [12]:
import numpy as np

In [13]:
from torch.utils.data.dataset import Dataset

In [14]:
from functools import lru_cache

In [15]:
from torchvision.transforms import Compose

In [16]:
from IPython.core.debugger import set_trace

In [17]:
from src.object_detection.data.coco import CocoDetectionDataset, make_id2category_map

In [18]:
from src.object_detection.utils.visualization import draw_bbox_xyhw, draw_rect_xywh_, draw_grid_

In [19]:
import torch

In [20]:
from src.object_detection.module import Module

In [60]:
from src.transform import to_rgb, to_torch, resize

In [28]:
def convert_to_torch(x):
    if not isinstance(x, np.ndarray):
        x = np.array(x)
    return torch.from_numpy(x)

In [29]:
def target_to_torch(target):
    if isinstance(target, dict):
        return {k: convert_to_torch(t) for k, t in target.items()}
    return convert_to_torch(target)

In [22]:
img_transform = Compose([
    to_torch,
])

In [30]:
coco_ds = CocoDetectionDataset(
    coco, 
    "resources/object_detection/train2014/",
    transform=to_torch,
    target_transform=target_to_torch,
    
)

In [31]:
num_classes = len(coco_ds.coco_dict["categories"])

In [32]:
#resnet = Resnet()
yolo = YoloV1(num_classes)

In [33]:
%%time
img1, annotation_dict1 = coco_ds[0]
img2, annotation_dict2 = coco_ds[5]

CPU times: user 21.3 ms, sys: 7.15 ms, total: 28.5 ms
Wall time: 26 ms


In [34]:
batch = [
    (img1, annotation_dict1),
    (img2, annotation_dict2),
]

In [35]:
def transpose(xss):
    return list(zip(*xss))

In [36]:
from torch.nn import functional as F

In [37]:
from collections import defaultdict

In [130]:
def pad_collate(batch):
    imgs, targets = transpose(batch)
    _, h_max, w_max = np.max([np.array(img.shape) for img in imgs], axis=0)
    padded_imgs = []
    
    padded_targets = defaultdict(list)
    for img, target in zip(imgs, targets):
        target = target_to_torch(target)
        _, h, w = img.shape
        pad_h, pad_w = h_max - h, w_max - w
        img = F.pad(img, (0, pad_w, 0, pad_h), "constant", 0)
        padded_imgs.append(img)
        if len(target):
            target["boxes"] = target["boxes"] * torch.from_numpy(np.array([h / h_max, w/w_max, h/h_max, w/w_max]))
        for name, v in target.items():
            padded_targets[name].append(v)
        
    
    imgs = torch.stack(padded_imgs)
    return imgs, padded_targets
    

In [131]:
from torch.utils.data.dataloader import DataLoader

In [132]:
ds_dl = DataLoader(coco_ds, collate_fn=pad_collate, batch_size=2, shuffle=True)

In [133]:
from src.object_detection.yolov1 import YoloV1Loss

In [134]:
yolov1_loss = YoloV1Loss()

In [135]:
from collections import defaultdict

In [136]:
def transpos_dict(dict_list):
    list_dict = defaultdict(list)
    for d in dict_list:
        for k, v in d.items():
            list_dict[k].append(v)
    return list_dict

In [137]:
def pad_images_by_hw(img, max_h, max_w):
    c, h, w = img.shape
    pad_h = max_h - h
    pad_w = max_w - w
    return F.pad(img, (0, pad_w, 0, pad_h), "constant", 0)

In [138]:
from src.object_detection.yolov1 import YoloV1Loss

In [139]:
yv1_loss = YoloV1Loss()

In [140]:
from light_torch.train import Trainer

In [141]:
from src.object_detection.module import ObjectDetectionModule

In [142]:
obj_detection = ObjectDetectionModule(yolo, yolov1_loss, coco_ds.id2name_map)

In [143]:
trainer = Trainer(obj_detection)

In [144]:
optimizer = torch.optim.SGD(obj_detection.parameters(), lr=0.001)

In [ ]:
trainer.fit(optimizer, 1, ds_dl)

Train epoch  001:   0%|          | 0/41392 [00:00<?, ?it/s]